## 1. Import

In [27]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import OneHotEncoder
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost.callback import EarlyStopping
import warnings
warnings.filterwarnings(action='ignore')

## 2. Data Load

In [6]:
# 학습/평가 데이터 로드
train_df = pd.read_csv('./data/train.csv').drop(columns=['UID'])
test_df = pd.read_csv('./data/test.csv').drop(columns=['UID'])

## 3. Pre-processing (전처리)

In [8]:
categorical_col = [
    '주거 형태',
    '현재 직장 근속 연수',
    '대출 목적',
    '대출 상환 기간'
]

# OneHotEncoder 초기화
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# 훈련 데이터에 대해 인코더 학습
encoder.fit(train_df[categorical_col])

# 훈련 데이터와 테스트 데이터 변환
train_encoded = encoder.transform(train_df[categorical_col])
test_encoded = encoder.transform(test_df[categorical_col])

# One-hot encoding 결과를 데이터프레임으로 변환
train_encoded_df = pd.DataFrame(train_encoded, columns=encoder.get_feature_names_out(categorical_col))
test_encoded_df = pd.DataFrame(test_encoded, columns=encoder.get_feature_names_out(categorical_col))

# 인코딩된 결과를 원래 데이터에 적용
train_df = pd.concat([train_df.drop(columns=categorical_col).reset_index(drop=True), train_encoded_df], axis=1)
test_df = pd.concat([test_df.drop(columns=categorical_col).reset_index(drop=True), test_encoded_df], axis=1)

## 4. Train / Validation Split (학습 데이터 분할)

In [9]:
X_train, X_val, y_train, y_val = train_test_split(
    train_df.drop(columns=['채무 불이행 여부']), 
    train_df['채무 불이행 여부'], 
    test_size=0.2, 
    random_state=42
)

## 5. Model Training (모델 학습)

In [29]:
# 예시 데이터 (실제 데이터로 교체하세요)
# X_train, y_train, X_val, y_val = ...

# ------------------------------
# Option 1: XGBClassifier (scikit-learn API)
# ------------------------------
# 이 방식은 early stopping 인자가 최신 버전에서는 문제가 발생할 수 있으므로,
# 기본 파라미터로 학습 후, 필요하다면 별도로 조기 종료 관련 처리를 해야 합니다.
model = XGBClassifier(
    n_estimators=100,        # 트리 개수
    max_depth=5,             # 최대 깊이
    learning_rate=0.15,      # 학습률
    random_state=42,
    use_label_encoder=False, # 경고 방지
    eval_metric="auc"        # 평가 지표
)

# 모델 학습 (early stopping 없이 진행)
model.fit(X_train, y_train)
print("XGBClassifier 학습 완료")


# ------------------------------
# Option 2: xgb.train() API with early stopping
# ------------------------------
# DMatrix로 데이터를 변환합니다.
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)
evals = [(dval, 'eval')]

# 파라미터 설정 (XGBClassifier와 동일한 설정으로 맞춰줍니다)
params = {
    'objective': 'binary:logistic',
    'max_depth': 5,
    'learning_rate': 0.15,
    'random_state': 42,
    'eval_metric': 'auc',
}

# xgb.train() 사용하여 조기 종료(early stopping) 적용
bst = xgb.train(params,
                dtrain,
                num_boost_round=10000,
                evals=evals,
                early_stopping_rounds=30)

print("xgb.train() 사용 학습 완료")
print("Best iteration:", bst.best_iteration)
print("Best score:", bst.best_score)


XGBClassifier 학습 완료
[0]	eval-auc:0.67949
[1]	eval-auc:0.69964
[2]	eval-auc:0.70016
[3]	eval-auc:0.70345
[4]	eval-auc:0.70659
[5]	eval-auc:0.70639
[6]	eval-auc:0.71079
[7]	eval-auc:0.71534
[8]	eval-auc:0.71936
[9]	eval-auc:0.72093
[10]	eval-auc:0.72245
[11]	eval-auc:0.72422
[12]	eval-auc:0.72550
[13]	eval-auc:0.72724
[14]	eval-auc:0.72821
[15]	eval-auc:0.72795
[16]	eval-auc:0.72853
[17]	eval-auc:0.72884
[18]	eval-auc:0.72887
[19]	eval-auc:0.73025
[20]	eval-auc:0.73043
[21]	eval-auc:0.73077
[22]	eval-auc:0.73113
[23]	eval-auc:0.73156
[24]	eval-auc:0.73284
[25]	eval-auc:0.73295
[26]	eval-auc:0.73348
[27]	eval-auc:0.73373
[28]	eval-auc:0.73408
[29]	eval-auc:0.73461
[30]	eval-auc:0.73517
[31]	eval-auc:0.73577
[32]	eval-auc:0.73554
[33]	eval-auc:0.73587
[34]	eval-auc:0.73533
[35]	eval-auc:0.73541
[36]	eval-auc:0.73582
[37]	eval-auc:0.73529
[38]	eval-auc:0.73537
[39]	eval-auc:0.73536
[40]	eval-auc:0.73547
[41]	eval-auc:0.73603
[42]	eval-auc:0.73571
[43]	eval-auc:0.73566
[44]	eval-auc:0.73587


## 6. Prediction

In [30]:
# 채무 불이행 '확률'을 예측합니다.
preds = model.predict_proba(test_df)[:,1]

## 7. Submission (제출 파일 생성)

In [31]:
submit = pd.read_csv('./data/sample_submission.csv')

# 결과 저장
submit['채무 불이행 확률'] = preds
submit.to_csv('./submission.csv', encoding='UTF-8-sig', index=False)